# 词向量到文本向量

## 双向RNN

RNN可以用来获取一段文本的单向量表示么？为什么？怎么做？

是的，循环神经网络（Recurrent Neural Network，RNN）可以被用来获取一段文本的向量表示。这是因为RNN能够处理序列数据，它可以在序列中的每个步骤（在这种情况下，就是每个词或词组）接收输入，并更新其隐藏状态。这个隐藏状态可以被看作是到目前为止序列的总结或表示。

以下是使用RNN获取文本向量表示的基本步骤：

1. **文本预处理**：这包括将文本切分成词或词组，然后将这些词或词组转换成向量。这可以通过词嵌入（如Word2Vec或GloVe）来完成。

2. **RNN模型**：然后，你可以将预处理的词向量序列输入到RNN模型中。RNN将从第一个词开始，逐步读取词向量，每读取一个词就更新其隐藏状态。

3. **获取文本向量**：最后，你可以使用RNN的最终隐藏状态作为整个文本的向量表示。这个隐藏状态是在读取完文本中所有词后得到的，因此它可以被看作是整个文本的总结。

这种方法的一个重要优点是，它可以捕获文本中的顺序信息，这是许多其他方法（如词袋模型）所不能做到的。然而，它也有一些缺点，比如计算复杂性较高，需要大量的训练数据，以及RNN可能会在处理长序列时遇到难度（这是因为梯度消失问题，即随着序列长度的增加，早期的信息可能会被忽视或遗忘）。

对于这个问题，长短期记忆网络（Long Short-Term Memory，LSTM）或门控循环单元（Gated Recurrent Unit，GRU）可以被用来替代标准的RNN，因为它们在处理长序列时更加有效。

In [1]:
import torch
import torch.nn as nn

class BiRNN(nn.Module):
    # word_dim为词向量长度，hidden_size为RNN隐状态维度
    def __init__(self, word_dim, hidden_size):
        super(BiRNN, self).__init__()
        # 双向GRU，输入的张量第一维是batch大小
        self.gru = nn.GRU(word_dim, hidden_size=hidden_size, bidirectional=True, batch_first=True) 

    # 输入x为batch组文本，长度seq_len，词向量长度为word_dim, 维度batch x seq_len x word_dim
    # 输出为文本向量，维度为batch x (2 x hidden_size)
    def forward(self, x):
        batch = x.shape[0]
        # output为每个单词对应的最后一层RNN的隐状态，维度为batch x seq_len x (2 x hidden_size)
        # last_hidden为最终的RNN状态，维度为2 x batch x hidden_size
        output, last_hidden = self.gru(x)
        return last_hidden.transpose(0,1).contiguous().view(batch, -1)  

batch = 10
seq_len = 20
word_dim = 50
hidden_size = 100
x = torch.randn(batch, seq_len, word_dim)
birnn = BiRNN(word_dim, hidden_size)
res = birnn(x)
print(res.shape) # torch.Size([10, 200])

torch.Size([10, 200])


## CNN与池化

为什么CNN可以用来获取一段文本的单向量表示？怎么做？
卷积神经网络（Convolutional Neural Networks，CNN）通常在计算机视觉任务中使用，但它们也可以应用于自然语言处理（NLP）任务，包括获取一段文本的向量表示。CNN在处理局部特征方面非常有效，这在图像和文本中都很有用。对于文本，这些局部特征可以是n-gram、词组或者短语。

以下是使用CNN获取文本向量表示的基本步骤：

1. **文本预处理**：首先，你需要将文本切分成词或词组，并将这些词或词组转换成向量。这可以通过词嵌入（如Word2Vec或GloVe）来完成。这将生成一个二维矩阵，其中每行是一个词向量。

2. **卷积层**：然后，你可以在这个矩阵上应用一维卷积。这实际上就是在词向量序列的滑动窗口上应用一个线性变换，然后通过一个非线性激活函数。这将生成一个新的二维矩阵，其中每行都是一个局部特征向量。

3. **池化层**：接下来，你可以在卷积层的输出上应用最大池化（max-pooling）操作。这将返回每个特征的最大值，相当于从每个滑动窗口中选出最重要的特征。这将生成一个特征向量，可以被看作是整个文本的向量表示。

以下是一个使用Python和PyTorch库的简单示例，它展示了如何使用一个简单的CNN模型来获取一段文本的向量表示：

```python
import torch
import torch.nn as nn
import torch.nn.functional as F

# 假设我们有一个10词的句子，每个词已经被转换为一个5维的向量
sentence = torch.randn(10, 5).unsqueeze(0)  # 添加一个批次维度

# 定义CNN模型
class TextCNN(nn.Module):
    def __init__(self):
        super(TextCNN, self).__init__()
        self.conv = nn.Conv1d(in_channels=5, out_channels=10, kernel_size=3)
        self.pool = nn.MaxPool1d(kernel_size=8)

    def forward(self, x):
        x = x.transpose(1, 2)  # 交换通道和长度维度以匹配PyTorch的期望输入
        x = F.relu(self.conv(x))
        x = self.pool(x)
        return x.flatten()  # 展平为一个向量

model = TextCNN()

# 将句子输入到CNN模型中
sentence_vector = model(sentence)
print(sentence_vector)
```

在这个例子中，我们定义了一个简单的CNN模型，它包括一个一维卷积层和一个最大池化层。在卷积层中，我们选择了一个尺寸为3的滑动窗口（也就是kernel_size=3），这相当于在每个窗口中看3个词。我们选择了10个输出通道，这意味着我们学习了10个不同的卷积核，每个都可以捕获不同的特征。然后，我们在卷积层的输出上应用了最大池化，选择了一个尺寸为8的窗口。这将从每个窗口中选择最重要的特征，从而生成一个10维的向量，可以被看作是整个句子的向量表示。

### 池化
池化（Pooling）是卷积神经网络（CNN）中常用的一种操作，它的目的是减少数据的空间大小（例如，减少图像的宽度和高度），从而减少计算量，同时也能控制过拟合。它通过在输入数据的滑动窗口上应用某种操作来实现，这个滑动窗口通常被称为池化窗口。

最常见的两种类型的池化操作是最大池化（Max Pooling）和平均池化（Average Pooling）：

1. **最大池化**：这种类型的池化操作在每个池化窗口上返回最大的值。这样可以确保即使在降低空间分辨率的同时，仍然可以保留最重要的特征（即最大的值）。这是最常用的池化操作类型，因为它在实践中通常能得到最好的结果。

2. **平均池化**：这种类型的池化操作在每个池化窗口上返回平均值。这意味着它会平等地考虑窗口中的所有值，而不仅仅是最大的值。这在某些情况下可能是有用的，但在实践中，它通常不如最大池化效果好。

在处理文本数据时，池化操作也可以被用来减少数据的大小。例如，在一个卷积神经网络（CNN）处理文本数据的情况下，你可能会在卷积层之后应用一个最大池化操作，这将为每个卷积核返回一个值，从而生成一个固定长度的向量，不论输入文本的长度如何。这个向量可以被看作是整个文本的向量表示。

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN_Maxpool(nn.Module):
    # word_dim为词向量长度，window_size为CNN窗口长度，output_dim为CNN输出通道数
    def __init__(self, word_dim, window_size, out_channels):
        super(CNN_Maxpool, self).__init__()
        # 1个输入通道，out_channels个输出通道，过滤器大小为window_size x word_dim
        self.cnn = nn.Conv2d(1, out_channels, (window_size, word_dim)) 

    # 输入x为batch组文本，长度seq_len，词向量长度为word_dim, 维度batch x seq_len x word_dim
    # 输出res为所有文本向量，每个向量维度为out_channels
    def forward(self, x):
        # 变成单通道，结果维度batch x 1 x seq_len x word_dim
        x_unsqueeze = x.unsqueeze(1) 
        # CNN, 结果维度batch x out_channels x new_seq_len x 1
        x_cnn = self.cnn(x_unsqueeze) 
        # 删除最后一维，结果维度batch x out_channels x new_seq_len
        x_cnn_result = x_cnn.squeeze(3) 
        # 最大池化，遍历最后一维求最大值，结果维度batch x out_channels
        res, _ = x_cnn_result.max(2)  
        return res


batch = 10
seq_len = 20
word_dim = 50
window_size = 3
out_channels = 100
x = torch.randn(batch, seq_len, word_dim)
cnn_maxpool = CNN_Maxpool(word_dim, window_size, out_channels)
res = cnn_maxpool(x)
print(res.shape) # torch.Size([10, 100])

torch.Size([10, 100])


## 含参加权求和

含参加权求和是一种捕获文本信息的策略，它可以用于获取一段文本的单向量表示。这种方法的关键思想是：不是给文本中的所有词赋予相同的权重，而是让模型学习每个词的权重。这种权重可能取决于词的内容、上下文等等。这种方法通常能够得到比简单的求和或平均更好的结果。

具体来说，我们可以定义一个含参的权重函数，这个函数接收一个词向量作为输入，输出一个标量权重。然后，我们可以用这个权重乘以词向量，然后对所有的词向量求和，从而得到整个文本的向量表示。

以下是一个使用Python和PyTorch库的简单示例，它展示了如何使用含参加权求和来获取一段文本的向量表示：

```python
import torch
import torch.nn as nn
import torch.nn.functional as F

# 假设我们有一个10词的句子，每个词已经被转换为一个5维的向量
sentence = torch.randn(10, 5)

# 定义权重函数，它是一个简单的前馈神经网络
weight_function = nn.Sequential(
    nn.Linear(5, 10),
    nn.Tanh(),
    nn.Linear(10, 1),
    nn.Softmax(dim=0)
)

# 计算每个词的权重
weights = weight_function(sentence)

# 使用权重对词向量进行加权求和
sentence_vector = torch.sum(weights * sentence, dim=0)
print(sentence_vector)
```

在这个例子中，我们定义了一个权重函数，它是一个简单的前馈神经网络，包括两个线性层和一个激活函数。我们将这个函数应用于每个词向量，然后使用softmax函数将输出转换为一个概率分布。这样，权重的总和就是1，这使得加权求和的结果是所有词向量的凸组合。然后，我们使用这些权重对词向量进行加权求和，从而得到整个句子的向量表示。

这只是一个基本的例子，实际上可能需要使用更复杂的权重函数，或者在计算权重时考虑更多的上下文信息。

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class WeightedSum(nn.Module):
    # 输入的词向量维度为word_dim
    def __init__(self, word_dim):
        super(WeightedSum, self).__init__()
        self.b = nn.Linear(word_dim, 1) # 参数张量

    # x：输入tensor,维度为batch x seq_len x word_dim
    # 输出res,维度是batch x word_dim
    def forward(self, x):
        # 内积得分，维度是batch x seq_len x 1
        scores = self.b(x)
        # softmax运算，结果维度是batch x seq_len x 1
        weights = F.softmax(scores, dim = 1) 
        # 用矩阵乘法实现加权和，结果维度是batch x word_dim x 1
        res = torch.bmm(x.transpose(1, 2), weights)  
        # 删除最后一维，结果维度是batch x word_dim 
        res = res.squeeze(2)  
        return res

batch = 10
seq_len = 20
word_dim = 50
x = torch.randn(batch, seq_len, word_dim)
weighted_sum = WeightedSum(word_dim)
res = weighted_sum(x)
print(res.shape) # torch.Size([10, 50])

torch.Size([10, 50])


# 自然语言理解 NLU

文本分类：有K个种类，模型需要判定将每个模输入文本分到哪个类别中。用于分类的模型被称为判定模型(discriminative model)

一个用于分类的深度学习网络通常由以下几个主要部分组成：

1. **编码器（Encoder）**：编码器的任务是将输入数据（例如，图像、文本或声音）转换为一种能够被深度学习模型理解和处理的形式。这通常涉及到一系列转换，如特征提取、降维和标准化等。对于图像数据，卷积神经网络（CNN）通常被用作编码器；对于文本数据，可以使用词嵌入模型（如Word2Vec或GloVe）进行编码，或者使用更复杂的模型如循环神经网络（RNN）、长短期记忆网络（LSTM）或Transformer。

2. **特征提取（Feature Extraction）**：在编码输入数据之后，深度学习模型将通过一系列隐藏层来提取和学习输入数据的有用特征。这些隐藏层可以是卷积层、全连接层、循环层、自注意力层等。每一层都会从其前一层接收输入，并产生一个新的、更高级别的特征表示。

3. **分类器（Classifier）**：在特征提取部分之后，深度学习模型通常会有一个或多个全连接层，这些层的目标是将学习到的特征映射到目标类别。这些层通常被视为分类器的一部分。最后一个全连接层的输出节点的数量通常等于目标类别的数量。

4. **输出层（Output Layer）**：最后，输出层通常会应用一个激活函数，如softmax，将最后一层的线性输出转化为目标类别的概率分布。这个概率分布可以用来进行预测和计算损失。

这只是一个一般的框架，不同的模型和任务可能会有不同的结构和组件。

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN_Maxpool(nn.Module):
    # word_dim为词向量长度，window_size为CNN窗口长度，output_dim为CNN输出通道数
    def __init__(self, word_dim, window_size, out_channels):
        super(CNN_Maxpool, self).__init__()
        # 1个输入通道，out_channels个输出通道，过滤器大小为window_size x word_dim
        self.cnn = nn.Conv2d(1, out_channels, (window_size, word_dim)) 

    # 输入x为batch组文本，长度seq_len，词向量长度为word_dim, 维度batch x seq_len x word_dim
    # 输出res为所有文本向量，每个向量维度为out_channels
    def forward(self, x):
        # 变成单通道，结果维度batch x 1 x seq_len x word_dim
        x_unsqueeze = x.unsqueeze(1) 
        # CNN, 结果维度batch x out_channels x new_seq_len x 1
        x_cnn = self.cnn(x_unsqueeze) 
        # 删除最后一维，结果维度batch x out_channels x new_seq_len
        x_cnn_result = x_cnn.squeeze(3) 
        # 最大池化，遍历最后一维求最大值，结果维度batch x out_channels
        res, _ = x_cnn_result.max(2)  
        return res

class NLUNet(nn.Module):
    # word_dim为词向量长度，window_size为CNN窗口长度，out_channels为CNN输出通道数，K为类别个数
    def __init__(self, word_dim, window_size, out_channels, K):
        super(NLUNet, self).__init__()
        # CNN和最大池化
        self.cnn_maxpool = CNN_Maxpool(word_dim, window_size, out_channels)  
        # 输出层为全连接层
        self.linear = nn.Linear(out_channels, K)     
    
    # x：输入tensor,维度为batch x seq_len x word_dim
    # 输出class_score,维度是batch x K
    def forward(self, x):
        # 文本向量，结果维度是batch x out_channels
        doc_embed = self.cnn_maxpool(x)  
        # 分类分数，结果维度是batch x K
        class_score = self.linear(doc_embed)     
        return class_score

K = 3     # 三分类
net = NLUNet(10, 3, 15, K)
# 共30个序列，每个序列长度5，词向量维度是10
x = torch.randn(30, 5, 10, requires_grad=True)   
# 30个真值分类，类别为0~K-1的整数 
y = torch.LongTensor(30).random_(0, K) 
optimizer = optim.SGD(net.parameters(), lr=1)  
# res大小为batch x K
res = net(x)
# PyTorch自带交叉熵函数，包含计算softmax
loss_func = nn.CrossEntropyLoss() 
loss = loss_func(res, y)
print('loss1 =', loss)
optimizer.zero_grad()
loss.backward()
optimizer.step()
res = net(x)
loss = loss_func(res, y)
print('loss2 =', loss) # loss2应该比loss1小


loss1 = tensor(1.0179, grad_fn=<NllLossBackward0>)
loss2 = tensor(0.7661, grad_fn=<NllLossBackward0>)


# 自然语言生成 NLG

可以实现生成任务的模型被称为生成模型(generative model)

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN_Maxpool(nn.Module):
    # word_dim为词向量长度，window_size为CNN窗口长度，output_dim为CNN输出通道数
    def __init__(self, word_dim, window_size, out_channels):
        super(CNN_Maxpool, self).__init__()
        # 1个输入通道，out_channels个输出通道，过滤器大小为window_size x word_dim
        self.cnn = nn.Conv2d(1, out_channels, (window_size, word_dim)) 

    # 输入x为batch组文本，长度seq_len，词向量长度为word_dim, 维度batch x seq_len x word_dim
    # 输出res为所有文本向量，每个向量维度为out_channels
    def forward(self, x):
        # 变成单通道，结果维度batch x 1 x seq_len x word_dim
        x_unsqueeze = x.unsqueeze(1) 
        # CNN, 结果维度batch x out_channels x new_seq_len x 1
        x_cnn = self.cnn(x_unsqueeze) 
        # 删除最后一维，结果维度batch x out_channels x new_seq_len
        x_cnn_result = x_cnn.squeeze(3) 
        # 最大池化，遍历最后一维求最大值，结果维度batch x out_channels
        res, _ = x_cnn_result.max(2)  
        return res


class NLGNet(nn.Module):
    # word_dim为词向量长度，window_size为CNN窗口长度，rnn_dim为RNN的状态维度，vocab_size为词汇表大小
    def __init__(self, word_dim, window_size, rnn_dim, vocab_size):
        super(NLGNet, self).__init__()
        # 单词编号与词向量对应参数矩阵
        self.embed = nn.Embedding(vocab_size, word_dim)  
        # CNN和最大池化        
        self.cnn_maxpool = CNN_Maxpool(word_dim, window_size, rnn_dim)
        # 单层单向GRU，batch是第0维
        self.rnn = nn.GRU(word_dim, rnn_dim, batch_first=True) 
        # 输出层为全连接层，产生一个位置每个单词的得分
        self.linear = nn.Linear(rnn_dim, vocab_size)     
    
    # x_id：输入文本的词编号,维度为batch x x_seq_len
    # y_id：真值输出文本的词编号,维度为batch x y_seq_len
    # 输出预测的每个位置每个单词的得分word_scores，维度是batch x y_seq_len x vocab_size
    def forward(self, x_id, y_id):
        # 得到输入文本的词向量，维度为batch x x_seq_len x word_dim
        x = self.embed(x_id) 
        # 得到真值输出文本的词向量，维度为batch x y_seq_len x word_dim
        y = self.embed(y_id) 
        # 输入文本向量，结果维度是batch x cnn_channels
        doc_embed = self.cnn_maxpool(x)
        # 输入文本向量作为RNN的初始状态，结果维度是1 x batch x y_seq_len x rnn_dim
        h0 = doc_embed.unsqueeze(0)
        # RNN后得到每个位置的状态，结果维度是batch x y_seq_len x rnn_dim
        rnn_output, _ = self.rnn(y, h0)
        # 每一个位置所有单词的分数，结果维度是batch x y_seq_len x vocab_size
        word_scores = self.linear(rnn_output)   
        return word_scores

vocab_size = 100                        # 100个单词
net = NLGNet(10, 3, 15, vocab_size)     # 设定网络 
# 共30个输入文本的词id，每个文本长度10
x_id = torch.LongTensor(30, 10).random_(0, vocab_size) 
# 共30个真值输出文本的词id，每个文本长度8
y_id = torch.LongTensor(30, 8).random_(0, vocab_size)
optimizer = optim.SGD(net.parameters(), lr=1) 
# 每个位置词表中每个单词的得分word_scores，维度为30 x 8 x vocab_size
word_scores = net(x_id, y_id)
# PyTorch自带交叉熵函数，包含计算softmax
loss_func = nn.CrossEntropyLoss()
# 将word_scores变为二维数组，y_id变为一维数组，计算损失函数值
loss = loss_func(word_scores[:,:-1,:].reshape(-1, vocab_size), y_id[:, 1:].reshape(-1))
print('loss1 =', loss)
optimizer.zero_grad()
loss.backward()
optimizer.step() 
word_scores = net(x_id, y_id)
loss = loss_func(word_scores[:,:-1,:].reshape(-1, vocab_size), y_id[:, 1:].reshape(-1))
print('loss2 =', loss) # loss2应该比loss1小

loss1 = tensor(4.6380, grad_fn=<NllLossBackward0>)
loss2 = tensor(4.6144, grad_fn=<NllLossBackward0>)


# 注意力

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

'''
  a: 被注意的的向量组，batch x m x dim 
  x: 进行注意力计算的向量组，batch x n x dim
'''
def attention(a, x):
    # 内积计算注意力分数，结果维度为batch x n x m
    scores = x.bmm(a.transpose(1, 2))
    # 对最后一维进行softmax
    alpha = F.softmax(scores, dim=-1)
    # 注意力向量，结果维度为batch x n x dim
    attended = alpha.bmm(a) 
    return attended

batch = 10
m = 20
n = 30
dim = 15
a = torch.randn(batch, m, dim)
x = torch.randn(batch, n, dim)
res = attention(a, x)
print(res.shape) # torch.Size([10, 30, 15])
